In [1]:
from PandOpt import pandopt
import pandas as pd
import numpy as np
import tqdm 
import pandas as pd
import numpy as np
import timeit
import functools


In [2]:
# Example usage
def simple_start(z):
    x = (z['A'] + z['B']) / z['C']
    x += z['B'] * z['D']
    return x / z['B']

def harder_func(z):
    x = (z['A'] + z['B']) / z['C']
    if x > 0:
        return x / z['B']
    x += z['B'] * z['D']
    return x * z['B']

def harder2_func(z):
    x = (z['A'] + z['B']) / z['C']
    if (k:=z['A']-z['C']) > (j:=z['B']/z['D']):
        return x / k
    x *= j
    return x - k if k > z['C'] else x + k

def harder3_func(z):
    g=lambda a, b: a if abs(a) > abs(b) else - 2 * (b**(-a))
    x = (z['A'] + z['B']) / z['C']
    if (k:=z['A']-z['C']) > (j:=g(z['B'],z['D'])):
        return j / k
    x *= j
    return x - k if k > z['C'] else x + k
        

In [3]:

def measure_performance(df, func, test_type, window_size=3):
    try:
        if test_type == "apply":
            operation = functools.partial(df.apply, func, axis=1)
        elif test_type == "rolling":
            operation = functools.partial(df.rolling(window=window_size).apply, func, raw=True)
        elif test_type == "groupby":
            operation = functools.partial(df.groupby('A').apply, func)
        else:
            raise ValueError("Invalid test type")

        start_time = timeit.default_timer()
        result = operation()
        elapsed_time = timeit.default_timer() - start_time
        return np.sum(result, axis=0), elapsed_time, None
    except Exception as e:
        return None, None, str(e)

def run_tests(data_size, test_funcs, agg_funcs, test_types):
    results = {}
    total_tests = (data_size - 1) * len(test_funcs + agg_funcs) * len(test_types)
    progress_bar = tqdm.tqdm(total=total_tests, desc="Running Tests", ncols=100)

    for test_num in range(1, data_size):
        df_size = int(10**test_num)
        for func in test_funcs + agg_funcs:
            for test_type in test_types:
                for test_iter in range(15):
                    pandas_df = pd.DataFrame(np.random.randn(df_size, 4), columns=['A', 'B', 'C', 'D']).astype(np.float32)
                    pandopt_df = pandopt(pandas_df)
                    pandas_checksum, pandas_time, pandas_error = measure_performance(pandas_df, func, test_type)
                    pandopt_checksum, pandopt_time, pandopt_error = measure_performance(pandopt_df, func, test_type)
    
                    key = f"Size: 10^{test_num}, Func: {func.__name__}, Test: {test_type} - {test_iter}"
                    results[key] = {
                        "Pandas Time (s)": pandas_time,
                        "Pandopt Time (s)": pandopt_time,
                        "Checksum Pandas": pandas_checksum,
                        "Checksum Pandopt": pandopt_checksum,
                        "Pandas Error": pandas_error,
                        "Pandopt Error": pandopt_error
                    }
    
                progress_bar.update(1)

    progress_bar.close()
    return pd.DataFrame.from_dict(results, orient='index')

# Run the tests
test_functions = [simple_start, harder_func, harder2_func, harder3_func]
agg_functions = []
test_types = ["apply"]

results_df = run_tests(data_size=7, test_funcs=test_functions, agg_funcs=agg_functions, test_types=test_types)
results_df

Running Tests:   0%|                                                         | 0/24 [00:00<?, ?it/s]

{} <function simple_start at 0x7f3f10e8da80>


Running Tests:   4%|██                                               | 1/24 [00:02<00:54,  2.38s/it]

[[ 1.1913959  -1.4160854  -0.49710423 -0.79077286 -0.21559821 -0.56292164
   0.35779637  0.14562789  0.04355303 -0.6078642 ]
 [-0.32915676 -0.19780855 -0.51590395  1.1348498   1.091438   -0.03214883
  -1.3134284  -0.01052305  1.3225116  -0.20755357]
 [ 0.8724099   1.5013642  -0.19882569  1.218727   -0.02052601 -0.07454454
  -0.46017504 -1.1258476  -2.478252    1.1302408 ]
 [ 1.3774569  -1.314558    1.4085132   0.57591784 -1.7834837  -0.74544495
   1.5888186   0.46047986 -0.84852886 -0.01667917]]
[-1.1099603 -4.028935  -6.7333546  0.5419473]
{} <function simple_start at 0x7f3f10e8da80>
[[ 0.52196586 -0.05863949 -0.66604275  3.0551147  -1.1634499  -0.17275524
   2.0706675  -1.0757242  -0.13815308 -1.2094935 ]
 [ 1.5110989  -0.5796262  -0.07275017  0.23985982  1.839315   -0.78656507
  -1.0679615  -0.47034198 -0.4728131   2.271437  ]
 [-1.5979162  -0.08871783  0.624515   -0.72790307 -0.48351052  1.0311408
   1.4403129  -2.266021   -1.5474056   0.43351176]
 [ 1.0136257   0.62720704  0.43180

Running Tests:   8%|████                                             | 2/24 [00:04<00:55,  2.51s/it]

...

<ast>:2: RuntimeWarning: invalid value encountered in scalar power
/tmp/ipykernel_911727/2896102278.py:22: RuntimeWarning: invalid value encountered in scalar power
  g=lambda a, b: a if abs(a) > abs(b) else - 2 * (b**(-a))


{} <function harder3_func at 0x7f3f10e8dda0>
[[-0.5014229   0.03975749 -0.6894234  ...  0.25471327  1.0349267
   1.0398576 ]
 [-0.10975806  0.58978665 -0.8765681  ...  1.7344114   0.37232557
  -0.0531483 ]
 [-0.2912393  -0.16245995  0.21914306 ...  1.3483715   0.68074596
   1.1081262 ]
 [ 2.021611   -0.6583084  -0.12444302 ... -0.7167852   1.237942
   0.09242729]]
[[-11.68594777]
 [         nan]
 [ -0.17403609]
 [         nan]]


<ast>:2: RuntimeWarning: invalid value encountered in scalar power
/tmp/ipykernel_911727/2896102278.py:22: RuntimeWarning: invalid value encountered in scalar power
  g=lambda a, b: a if abs(a) > abs(b) else - 2 * (b**(-a))
<ast>:2: RuntimeWarning: invalid value encountered in scalar power
Running Tests: 100%|████████████████████████████████████████████████| 24/24 [12:12<00:00, 30.54s/it]

{} <function harder3_func at 0x7f3f10e8dda0>
[[ 0.56289434 -1.9875224   0.24797449 ...  0.28231013  1.4870409
  -0.3001791 ]
 [ 0.9852874  -0.31637758  0.03794917 ...  0.30307725 -0.35820484
  -1.0771637 ]
 [ 1.1189965  -0.33170986  0.86621165 ...  1.4240428  -0.18498625
  -0.18145421]
 [ 0.05468196 -0.05848348 -0.76874167 ...  0.9175182  -2.444442
   0.06508839]]
[[-6.31120062]
 [-0.33396474]
 [        nan]
 [        nan]]


,Pandas Time (s),Pandopt Time (s),Checksum Pandas,Checksum Pandopt,Pandas Error,Pandopt Error
"Size: 10^1, Func: simple_start, Test: apply - 0",0.000469,None,-2.810120e+02,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^1, Func: simple_start, Test: apply - 1",0.000398,None,-1.176684e+01,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^1, Func: simple_start, Test: apply - 2",0.000270,None,-1.143776e+01,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^1, Func: simple_start, Test: apply - 3",0.000258,None,-3.933148e+01,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^1, Func: simple_start, Test: apply - 4",0.000264,None,3.309672e+01,None,None,"Shape of passed values is (4, 1), indices impl..."
...,...,...,...,...,...,...
"Size: 10^6, Func: harder3_func, Test: apply - 10",13.050153,None,-1.745620e+06,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^6, Func: harder3_func, Test: apply - 11",13.306674,None,-2.322734e+05,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^6, Func: harder3_func, Test: apply - 12",13.049457,None,-2.570317e+06,None,None,"Shape of passed values is (4, 1), indices impl..."
"Size: 10^6, Func: harder3_func, Test: apply - 13",13.157856,None,3.825290e+06,None,None,"Shape of passed values is (4, 1), indices impl..."


In [4]:
results_df['time reduction'] = results_df["Pandopt Time (s)"] / results_df["Pandas Time (s)"] - 1
results_df['performance multiplicator'] = results_df["Pandas Time (s)"] / results_df["Pandopt Time (s)"] - 1
results_df

,Pandas Time (s),Pandopt Time (s),Checksum Pandas,Checksum Pandopt,Pandas Error,Pandopt Error,time reduction,performance multiplicator
"Size: 10^1, Func: simple_start, Test: apply - 0",0.000469,None,-2.810120e+02,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^1, Func: simple_start, Test: apply - 1",0.000398,None,-1.176684e+01,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^1, Func: simple_start, Test: apply - 2",0.000270,None,-1.143776e+01,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^1, Func: simple_start, Test: apply - 3",0.000258,None,-3.933148e+01,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^1, Func: simple_start, Test: apply - 4",0.000264,None,3.309672e+01,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
...,...,...,...,...,...,...,...,...
"Size: 10^6, Func: harder3_func, Test: apply - 10",13.050153,None,-1.745620e+06,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^6, Func: harder3_func, Test: apply - 11",13.306674,None,-2.322734e+05,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^6, Func: harder3_func, Test: apply - 12",13.049457,None,-2.570317e+06,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN
"Size: 10^6, Func: harder3_func, Test: apply - 13",13.157856,None,3.825290e+06,None,None,"Shape of passed values is (4, 1), indices impl...",NaN,NaN


In [5]:
results_df.to_csv('benchmark.csv')